# Systems Recruitment Task - Cryptography

## Given text is:
- d3ZucXN0b2tib2xlamp5ZW5zdnlicGpsa3VhcGx2 
- b2dzd2xmcndwb2JmY2J4dmdtZGZseGp1dnZuaGZ0amFiZ2M=
- YW9sdWxlYXJsZnB6anlmd2F2bnlod29m
- aGZyZ3VyeHJsZ2pyYWds
- Z3JlYXRzdGFydA==


In [1]:
cypher_text = ["d3ZucXN0b2tib2xlamp5ZW5zdnlicGpsa3VhcGx2", 
               "b2dzd2xmcndwb2JmY2J4dmdtZGZseGp1dnZuaGZ0amFiZ2M=",
               "YW9sdWxlYXJsZnB6anlmd2F2bnlod29m",
               "aGZyZ3VyeHJsZ2pyYWds",
               "Z3JlYXRzdGFydA=="]

#### Looking at the last line, it seems to be base64 encrypted (due to the padding in the end)

### Decrypting base64 encryption using custom decryptor

In [2]:
#to help in making the base64 index, hence helping in substituting characters with respective 6-bit binary representation
def int_to_bin64(num):
    result = ''
    count = 0
    while num > 0:
        result += str(num%2)
        num//=2
        count+=1
    result = '0'*(6-count) + result[::-1]
    return result
    

In [3]:
#creating the base64 index
base64index = {}
for i in range (26):
    base64index[chr(65+i)] = int_to_bin64(i)
for i in range (26):
    base64index[chr(97+i)] = int_to_bin64(26+i)
for i in range (10):
    base64index[str(i)] = int_to_bin64(52+i)
base64index['+'] = int_to_bin64(62)
base64index['/'] = int_to_bin64(63)
base64index

{'A': '000000',
 'B': '000001',
 'C': '000010',
 'D': '000011',
 'E': '000100',
 'F': '000101',
 'G': '000110',
 'H': '000111',
 'I': '001000',
 'J': '001001',
 'K': '001010',
 'L': '001011',
 'M': '001100',
 'N': '001101',
 'O': '001110',
 'P': '001111',
 'Q': '010000',
 'R': '010001',
 'S': '010010',
 'T': '010011',
 'U': '010100',
 'V': '010101',
 'W': '010110',
 'X': '010111',
 'Y': '011000',
 'Z': '011001',
 'a': '011010',
 'b': '011011',
 'c': '011100',
 'd': '011101',
 'e': '011110',
 'f': '011111',
 'g': '100000',
 'h': '100001',
 'i': '100010',
 'j': '100011',
 'k': '100100',
 'l': '100101',
 'm': '100110',
 'n': '100111',
 'o': '101000',
 'p': '101001',
 'q': '101010',
 'r': '101011',
 's': '101100',
 't': '101101',
 'u': '101110',
 'v': '101111',
 'w': '110000',
 'x': '110001',
 'y': '110010',
 'z': '110011',
 '0': '110100',
 '1': '110101',
 '2': '110110',
 '3': '110111',
 '4': '111000',
 '5': '111001',
 '6': '111010',
 '7': '111011',
 '8': '111100',
 '9': '111101',
 '+': '1

In [4]:
#to convert binary numbers to their decimal equivalent
def bin_to_dec(num):
    result = 0
    count = 0
    while num > 0:
        result += (num%10)*(2**count)
        num//=10
        count+=1
    return result

In [5]:
#example to show that the above function works
bin_to_dec(1100011)

99

In [6]:
#the main decrypting function for base64
def base64decode(text):
    bin_text = ''
    for i in text:
        if i != '=':
            bin_text += base64index[i]
    k=0
    result = ''
    while k <= len(bin_text)-8:
        num = int(bin_text[k:k+8])
        num = (bin_to_dec(num))
        result += chr(num)
        k+=8
    return result

In [7]:
#decrypting the encrypted text
for i in range(5):
    temp = base64decode(cypher_text[i])
    cypher_text[i]  = temp
cypher_text

['wvnqstokbolejjyensvybpjlkuaplv',
 'ogswlfrwpobfcbxvgmdflxjuvvnhftjabgc',
 'aolulearlfpzjyfwavnyhwof',
 'hfrgurxrlgjragl',
 'greatstart']

#### The encryption was base64 after all, as we are supposedly off to a "great start".

#### e4 seems to be a Caesar cipher with offset 13

In [8]:
#creating dictionaries to retrieve alphabets
dict1 = {}
dict2 = {}
alphabet = 'abcdefghijklmnopqrstuvwxyz'
for i in range(26):
    dict1[i+1] = alphabet[i]
    dict2[alphabet[i]] = i+1

In [9]:
#function to decrypt caesar cipher
def caesar(message, offset):
    decrypted = ''
    for i in message:
        num = dict2[i] - offset
        if num <= 0:
            num+=26
        decrypted += dict1[num]
    return decrypted

In [10]:
#decrypting the first 4 strings
for i in range(4):
    temp = caesar(cypher_text[i], 13)
    cypher_text[i] = temp

In [11]:
cypher_text

['jiadfgbxobyrwwlrafilocwyxhncyi',
 'btfjysejcbospokitzqsykwhiiausgwnotp',
 'nbyhyrneyscmwlsjnialujbs',
 'usethekeytwenty',
 'greatstart']

#### e4 indeed is a Caesar cipher

#### e3, from the hints and given logic, is again a Caesar cipher, but with offset 20

In [12]:
#decrypting the first 3 strings
for i in range(3):
    temp = caesar(cypher_text[i], 20)
    cypher_text[i] = temp
cypher_text

['pogjlmhduhexccrxgloruicedntieo',
 'hzlpeykpihuyvuqozfwyeqcnoogaymctuzv',
 'thenextkeyiscryptography',
 'usethekeytwenty',
 'greatstart']

#### Looking at the hint given, e2 seems to be a substitution cipher. From the given logic, it seems that e2 is a Vigenere cipher

In [13]:
#decrypting function for vignere cipher
def vignere_decode(message, key):
    key_len = len(key)
    msg_len = len(message)
    decrypted = ''
    for i in range(msg_len):
        num = (dict2[message[i]] - dict2[key[i%key_len]] + 26) % 26
        decrypted += dict1[num+1]
    return decrypted

In [14]:
#decrypting the first 2 strings
vignere_key = 'cryptography'
for i in range(2):
    cypher_text[i] = vignere_decode(cypher_text[i], vignere_key)
cypher_text

['nxiusybmusxzaltinxiautvgbwvtla',
 'finalkeyisnatdszgrqhebvpmxilfywcuko',
 'thenextkeyiscryptography',
 'usethekeytwenty',
 'greatstart']

#### e2 is indeed a Vigenere cipher. 

#### Looking at e1, it again seems to be a substitution cipher. 

In [15]:
key = 'natdszgrqhebvpmxilfywcuko'
print(len(key))

25


### The number of letters in the key is 25, and is missing a 'j' as well. This is usually a characteristic of a Playfair cipher, where we use a 5x5 matrix, with each element being one of the English alphabets (excluding 'j')

In [16]:
#initializing the playfair cipher square to help in decrypting
def playfair_init(key):
    playfair_sq = {}
    playfair_retrieve = {}
    k=0
    i=0
    j=0
    alphabet = 'abcdefghiklmnopqrstuvwxyz'
    bool_alph = {}
    for ch in alphabet:
        bool_alph[ch] = False
    while k < len(key):
        if bool_alph[key[k]] == False:
            playfair_sq[key[k]] = (k//5,k%5)
            playfair_retrieve[(k//5,k%5)] = key[k]
            bool_alph[key[k]] = True
        k+=1    
    for ch, bool_val in bool_alph.items():
            if bool_val == False:
                playfair_sq[ch] = (k//5, k%5)
                playfair_retrieve[(k//5, k%5)] = ch
                k+=1
        
    return playfair_sq, playfair_retrieve


In [17]:
#main playfair decryption function
def playfair_decode(message, square, retrieve):
    msg_split = []
    i = 0
    if len(message)%2: message+='x'
    while i < len(message):
        temp = ''
        if message[i] == message[i+1]:
            temp += (message[i]+'x')
        else:
            temp += (message[i]+message[i+1])
        i+=2
        msg_split.append(temp)
    result = ''
    for pair in msg_split:
        if square[pair[0]][0] == square[pair[1]][0]:
            result += retrieve[(square[pair[0]][0], (square[pair[0]][1]+4)%5)]
            result += retrieve[(square[pair[0]][0], (square[pair[1]][1]+4)%5)]
        elif square[pair[0]][1] == square[pair[1]][1]:
            result += retrieve[((square[pair[0]][0]+4)%5), square[pair[0]][1]]
            result += retrieve[((square[pair[1]][0]+4)%5), square[pair[0]][1]]
        else:
            result += retrieve[(square[pair[0]][0], square[pair[1]][1])]
            result += retrieve[(square[pair[1]][0], square[pair[0]][1])]
    return result

In [18]:
playfair_sq, playfair_retrieve = playfair_init(key)

In [19]:
#deciphering the first string
cypher_text[0] = playfair_decode(cypher_text[0], playfair_sq, playfair_retrieve)

In [20]:
print(cypher_text)

['welcomepotentialwebclubrecruit', 'finalkeyisnatdszgrqhebvpmxilfywcuko', 'thenextkeyiscryptography', 'usethekeytwenty', 'greatstart']


### The message has been decoded. 

### Summary of encryption algorithms used here:
- e1: Playfair cipher
- e2: Vigenere cipher
- e3: Caesar cipher (offset 20)
- e4: Caesar cipher (offset 13)
- e5: Base64 encrpytion